In [70]:
# hddm
import hddm
import kabuki
import pymc

# design matrix
from patsy import dmatrix

# data analysis
import pandas as pd
import numpy as np

# beyasian modeling
import bambi as bmb

# plot
import seaborn as sns
import matplotlib.pyplot as plt
import arviz as az
import ptitprince as pt

# parallel processing related
from p_tqdm import p_map
from functools import partial
from joblib import Parallel,delayed

# rmANOVA
from statsmodels.stats.anova import AnovaRM

# search file
import glob

# system
import os,sys,time

In [71]:
# add `script` to the python path so that we can import the scripts
# scripts_dir = '/home/jovyan/work/scripts'
scripts_dir = '/home/jovyan/scripts'
sys.path.append(scripts_dir)

from HDDMarviz import HDDMarviz
from InferenceDataFromHDDM import InferenceDataFromHDDM
from plot_ppc_by_cond import plot_ppc_by_cond

In [72]:
df = hddm.load_csv(hddm.__path__[0]+'/examples/cavanagh_theta_nn.csv')

In [73]:
df

,subj_idx,stim,rt,response,theta,dbs,conf
0,0,LL,1.210,1.0,0.656275,1,HC
1,0,WL,1.630,1.0,-0.327889,1,LC
2,0,WW,1.030,1.0,-0.480285,1,HC
3,0,WL,2.770,1.0,1.927427,1,LC
4,0,WW,1.140,0.0,-0.213236,1,HC
...,...,...,...,...,...,...,...
3983,13,LL,1.450,0.0,-1.237166,0,HC
3984,13,WL,0.711,1.0,-0.377450,0,LC
3985,13,WL,0.784,1.0,-0.694194,0,LC
3986,13,LL,2.350,0.0,-0.546536,0,HC


In [74]:
def ms1_issue(id, df=None, samples=None, burn=None, thin=1,save_name='ms1'):
    '''
    id: the id of cpu thread
    df: the input data
    samples: number of samples for MCMC
    burn: number of burn-in of MCMC
    thin: number of thin
    save_name: prefix of file name when saving the model object
    '''
    print('running chain {:d} for model {}'.format(id, save_name))
    # the database file
    dbname = save_name + '_chain_%i.db'%id
    # the model object store here
    mname = save_name + '_chain_%i'%id

    v_reg = {'model':'v~1','link_func':lambda x:x}
    z_reg = {'model':'z~1','link_func':lambda x:x}
    t_reg = {'model':'t~1','link_func':lambda x:x}
    a_reg = {'model':'a~1','link_func':lambda x:x}
    reg_descr = [v_reg, 
                 z_reg,
                 t_reg,
                 a_reg,]
    m = hddm.HDDMRegressor(df, 
                           reg_descr, 
                           include=['z'],
                           group_only_regressors=False,
                           keep_regressor_trace=True
                          )
    # find the starting point
    m.find_starting_values()
    # MCMC sample
    m.sample(samples, 
             burn=burn, 
             thin=thin, 
             dbname=dbname,
             db='pickle')
    # save
    m.save(mname)

    return m

In [75]:
msires1 = p_map(partial(ms1_issue,df=df,samples=100,burn=10,save_name='ms1_issue'),range(4))

  0%|          | 0/4 [00:00<?, ?it/s]

In [6]:
%%time
InfData_msi = InferenceDataFromHDDM(msires, nppc = 10, save_name="msi")

start converting msi


  0%|          | 0/4 [00:00<?, ?it/s]

started to generate posterior predicitves
started to generate posterior predicitves
started to generate posterior predicitves
started to generate posterior predicitves
 [----------       28%                  ] 4 of 14 complete in 7.2 sec[----------       28%                  ] 4 of 14 complete in 7.3 sec[----------       28%                  ] 4 of 14 complete in 7.4 sec[----------       28%                  ] 4 of 14 complete in 7.5 sec[-------------    35%                  ] 5 of 14 complete in 14.2 sec[-------------    35%                  ] 5 of 14 complete in 14.6 sec[-------------    35%                  ] 5 of 14 complete in 14.9 sec[-------------    35%                  ] 5 of 14 complete in 14.7 sec[---------------- 42%                  ] 6 of 14 complete in 20.8 sec[---------------- 42%                  ] 6 of 14 complete in 21.4 sec[---------------- 42%                  ] 6 of 14 complete in 21.8 sec[---------------- 42%                  ] 6 of 14 complete in 21.6 sec[------

  0%|          | 0/4 [00:00<?, ?it/s]

started to generate posterior predicitves
started to generate posterior predicitves
started to generate posterior predicitves
started to generate posterior predicitves
 [----------       28%                  ] 4 of 14 complete in 4.5 sec[----------       28%                  ] 4 of 14 complete in 4.4 sec[----------       28%                  ] 4 of 14 complete in 4.5 sec[----------       28%                  ] 4 of 14 complete in 4.6 sec[-------------    35%                  ] 5 of 14 complete in 9.0 sec[-------------    35%                  ] 5 of 14 complete in 8.9 sec[-------------    35%                  ] 5 of 14 complete in 8.9 sec[-------------    35%                  ] 5 of 14 complete in 9.2 sec[---------------- 42%                  ] 6 of 14 complete in 13.3 sec[---------------- 42%                  ] 6 of 14 complete in 13.4 sec[---------------- 42%                  ] 6 of 14 complete in 13.5 sec[---------------- 42%                  ] 6 of 14 complete in 13.7 sec[----------

In [9]:
msires = Parallel(n_jobs = 4)(delayed(ms1_issue)(id = i, df = df, samples=100,burn = 10,thin=2, save_name = 'msi' ) for i in range(4))

In [10]:
%%time
InfData_msi = InferenceDataFromHDDM(msires, nppc = 10, save_name="msi2")

start converting msi2


  0%|          | 0/4 [00:00<?, ?it/s]

started to generate posterior predicitves
started to generate posterior predicitves
started to generate posterior predicitves
started to generate posterior predicitves
 [----------       28%                  ] 4 of 14 complete in 7.2 sec[----------       28%                  ] 4 of 14 complete in 7.2 sec[----------       28%                  ] 4 of 14 complete in 7.4 sec[----------       28%                  ] 4 of 14 complete in 7.2 sec[-------------    35%                  ] 5 of 14 complete in 14.5 sec[-------------    35%                  ] 5 of 14 complete in 14.6 sec[-------------    35%                  ] 5 of 14 complete in 14.5 sec[-------------    35%                  ] 5 of 14 complete in 15.0 sec[---------------- 42%                  ] 6 of 14 complete in 21.2 sec[---------------- 42%                  ] 6 of 14 complete in 21.2 sec[---------------- 42%                  ] 6 of 14 complete in 21.5 sec[---------------- 42%                  ] 6 of 14 complete in 22.0 sec[------

  0%|          | 0/4 [00:00<?, ?it/s]

started to generate posterior predicitves
started to generate posterior predicitves
started to generate posterior predicitves
started to generate posterior predicitves
 [----------       28%                  ] 4 of 14 complete in 2.3 sec[----------       28%                  ] 4 of 14 complete in 2.3 sec[----------       28%                  ] 4 of 14 complete in 2.4 sec[----------       28%                  ] 4 of 14 complete in 2.3 sec[-------------    35%                  ] 5 of 14 complete in 4.5 sec[-------------    35%                  ] 5 of 14 complete in 4.6 sec[-------------    35%                  ] 5 of 14 complete in 4.6 sec[-------------    35%                  ] 5 of 14 complete in 4.7 sec[---------------- 42%                  ] 6 of 14 complete in 6.8 sec[---------------- 42%                  ] 6 of 14 complete in 6.9 sec[---------------- 42%                  ] 6 of 14 complete in 6.8 sec[---------------- 42%                  ] 6 of 14 complete in 7.1 sec[--------------

In [43]:
# set number of subjects and number of trials per level for the simulated experiment.
n_subjects = 10
trials_per_level = 50

# set the group parameter
# intercept and slope of v
v_int = 1
v_sig = 0.05
# intercept and slope of z
z_int = 0.5
z_sig = 0.02
# intercept of a
a_int = 1.4
a_sig = 0.1
# intercept of t
t_int = 0.2
t_sig = 0.05

In [44]:
# set the subject parameter
# v
v_err_subj = np.random.normal(0,v_sig,n_subjects)
# z
z_err_subj = np.random.normal(0,z_sig,n_subjects)
# a
a_err_subj = np.random.normal(0,a_sig,n_subjects)
# t
t_err_subj = np.random.normal(0,t_sig,n_subjects)

# dataframe
df = pd.DataFrame()
param = pd.DataFrame()
for i in range(n_subjects):
    
    # v of subject 
    v  = v_int + v_err_subj[i]
    # z of subject
    z  = z_int + z_err_subj[i]
    # the default parameter a
    a  = a_int + a_err_subj[i]
    # the default parameter t
    t  = t_int + t_err_subj[i]
    
    # set up parameters of DDM for four levels 
    level = {'v':v, 'a':a, 't':t, 'z':z, 'sv':0, 'sz':0, 'st':0}
    
    # generate simulated data
    np.random.seed(123)
    data, params = hddm.generate.gen_rand_data({'level':level},
                                                 size = trials_per_level
                                                  )
    # data
    data['subj_idx'] = i
    df = df.append(data)
    
    # params
    # create dataframe
    params = pd.DataFrame([params])
    # add subject
    params['subj_idx'] = i
    # combine params
    param = param.append(params)
# reset the index
param = param.reset_index(drop=True)

In [46]:
df

,rt,response,subj_idx,condition
0,0.496124,1.0,0,level
1,0.257124,1.0,0,level
2,0.228124,1.0,0,level
3,0.388124,1.0,0,level
4,0.518124,1.0,0,level
...,...,...,...,...
45,0.246374,1.0,9,level
46,0.379374,1.0,9,level
47,1.514374,1.0,9,level
48,0.402374,1.0,9,level


In [47]:
msires = p_map(partial(ms1_issue,df=df,samples=100,burn=10,save_name='ms1_issue'),range(4))

  0%|          | 0/4 [00:00<?, ?it/s]

In [48]:
%%time
InfData_msi = InferenceDataFromHDDM(msires, nppc = 10, save_name="msi4")

start converting msi4


  0%|          | 0/4 [00:00<?, ?it/s]

started to generate posterior predicitves


NotImplementedError: Supply a grouping so that at most 1 observed node codes for each group.

started to generate posterior predicitves
started to generate posterior predicitves
started to generate posterior predicitves
started to generate posterior predicitves
started to generate posterior predicitves
started to generate posterior predicitves
started to generate posterior predicitves
running chain 0 for model ms1_issue
Adding these covariates:running chain 1 for model ms1_issue

['v_Intercept']
Adding these covariates:Adding these covariates:

['z_Intercept']
['v_Intercept']
Adding these covariates:
['t_Intercept']Adding these covariates:

Adding these covariates:['z_Intercept']
running chain 2 for model ms1_issue
['a_Intercept']

Adding these covariates:
['t_Intercept']
Adding these covariates:Adding these covariates:

['v_Intercept']['a_Intercept']

running chain 3 for model ms1_issueAdding these covariates:
['z_Intercept']

Adding these covariates:Adding these covariates:

['v_Intercept']['t_Intercept']

Adding these covariates:Adding these covariates:

['a_Intercept']['z_I

/opt/conda/lib/python3.8/site-packages/scipy/optimize/optimize.py:2215: RuntimeWarning: invalid value encountered in double_scalars
  tmp2 = (x - v) * (fx - fw)
/opt/conda/lib/python3.8/site-packages/scipy/optimize/optimize.py:2215: RuntimeWarning: invalid value encountered in double_scalars
  tmp2 = (x - v) * (fx - fw)
/opt/conda/lib/python3.8/site-packages/scipy/optimize/optimize.py:2215: RuntimeWarning: invalid value encountered in double_scalars
  tmp2 = (x - v) * (fx - fw)
/opt/conda/lib/python3.8/site-packages/scipy/optimize/optimize.py:2215: RuntimeWarning: invalid value encountered in double_scalars
  tmp2 = (x - v) * (fx - fw)


 [-                 3%                  ] 3 of 100 complete in 0.5 sec[-                 3%                  ] 3 of 100 complete in 0.5 sec[-                 3%                  ] 3 of 100 complete in 0.5 sec[-                 4%                  ] 4 of 100 complete in 0.7 sec[-                 5%                  ] 5 of 100 complete in 1.0 sec[-                 5%                  ] 5 of 100 complete in 1.0 sec[-                 5%                  ] 5 of 100 complete in 1.0 sec[--                6%                  ] 6 of 100 complete in 1.3 sec[--                7%                  ] 7 of 100 complete in 1.6 sec[--                7%                  ] 7 of 100 complete in 1.6 sec[--                7%                  ] 7 of 100 complete in 1.6 sec[---               8%                  ] 8 of 100 complete in 1.9 sec[---               9%                  ] 9 of 100 complete in 2.2 sec[---               9%                  ] 9 of 100 complete in 2.2 sec[---               9%            

In [49]:
msires = Parallel(n_jobs = 4)(delayed(ms1_issue)(id = i, df = df, samples=100,burn = 10,thin=2, save_name = 'msi' ) for i in range(4))

In [50]:
%%time
InfData_msi = InferenceDataFromHDDM(msires, nppc = 10, save_name="msi3")

start converting msi3


  0%|          | 0/4 [00:00<?, ?it/s]

NotImplementedError: Supply a grouping so that at most 1 observed node codes for each group.

In [53]:
level = ['A']*25+['B']*25

In [57]:
for i in range(10):
    df.loc[df['subj_idx']== i ,'condition'] = level 

In [59]:
msires = p_map(partial(ms1_issue,df=df,samples=100,burn=10,save_name='ms1_issue'),range(4))

  0%|          | 0/4 [00:00<?, ?it/s]

In [60]:
%%time
InfData_msi = InferenceDataFromHDDM(msires, nppc = 10, save_name="msi4")

start converting msi4


  0%|          | 0/4 [00:00<?, ?it/s]

started to generate posterior predicitves


NotImplementedError: Supply a grouping so that at most 1 observed node codes for each group.

started to generate posterior predicitves
started to generate posterior predicitves
started to generate posterior predicitves
started to generate posterior predicitves
started to generate posterior predicitves
started to generate posterior predicitves
started to generate posterior predicitves
running chain 0 for model ms1_issue
Adding these covariates:
['v_Intercept']
Adding these covariates:
['z_Intercept']
Adding these covariates:
['t_Intercept']
Adding these covariates:
['a_Intercept']
running chain 1 for model ms1_issue
Adding these covariates:
['v_Intercept']
Adding these covariates:
['z_Intercept']
Adding these covariates:
['t_Intercept']
Adding these covariates:
['a_Intercept']
running chain 2 for model ms1_issue
Adding these covariates:
['v_Intercept']
Adding these covariates:
['z_Intercept']
Adding these covariates:
['t_Intercept']
Adding these covariates:
['a_Intercept']
running chain 3 for model ms1_issue
Adding these covariates:
['v_Intercept']
Adding these covariates:
['z_

/opt/conda/lib/python3.8/site-packages/scipy/optimize/optimize.py:2215: RuntimeWarning: invalid value encountered in double_scalars
  tmp2 = (x - v) * (fx - fw)
/opt/conda/lib/python3.8/site-packages/scipy/optimize/optimize.py:2215: RuntimeWarning: invalid value encountered in double_scalars
  tmp2 = (x - v) * (fx - fw)
/opt/conda/lib/python3.8/site-packages/scipy/optimize/optimize.py:2215: RuntimeWarning: invalid value encountered in double_scalars
  tmp2 = (x - v) * (fx - fw)
/opt/conda/lib/python3.8/site-packages/scipy/optimize/optimize.py:2215: RuntimeWarning: invalid value encountered in double_scalars
  tmp2 = (x - v) * (fx - fw)


 [-                 3%                  ] 3 of 100 complete in 0.8 sec[-                 3%                  ] 3 of 100 complete in 0.8 sec[-                 3%                  ] 3 of 100 complete in 0.8 sec[-                 3%                  ] 3 of 100 complete in 0.9 sec[-                 5%                  ] 5 of 100 complete in 1.4 sec[-                 5%                  ] 5 of 100 complete in 1.4 sec[-                 5%                  ] 5 of 100 complete in 1.5 sec[-                 5%                  ] 5 of 100 complete in 1.5 sec[--                7%                  ] 7 of 100 complete in 2.1 sec[--                7%                  ] 7 of 100 complete in 2.1 sec[--                7%                  ] 7 of 100 complete in 2.2 sec[--                7%                  ] 7 of 100 complete in 2.2 sec[---               9%                  ] 9 of 100 complete in 2.7 sec[---               9%                  ] 9 of 100 complete in 2.7 sec[---               9%            

In [61]:
msires = Parallel(n_jobs = 4)(delayed(ms1_issue)(id = i, df = df, samples=100,burn = 10,thin=2, save_name = 'msi' ) for i in range(4))

In [62]:
%%time
InfData_msi = InferenceDataFromHDDM(msires, nppc = 10, save_name="msi3")

start converting msi3


  0%|          | 0/4 [00:00<?, ?it/s]

NotImplementedError: Supply a grouping so that at most 1 observed node codes for each group.

In [85]:
for a, b in msires[0].iter_observeds():
    node = b['node']
node.value

,rt
0,0.549374
1,0.267374
2,0.231374
3,0.424374
4,0.574374
5,0.343374
6,1.424374
7,0.537374
8,0.377374
9,0.325374


In [83]:
for a, b in msires1[0].iter_observeds():
    node = b['node']

In [84]:
node.value

,rt
3714,1.500
3715,0.929
3716,1.880
3717,-1.180
3718,1.810
...,...
3983,-1.450
3984,0.711
3985,0.784
3986,-2.350


In [91]:
df = df.reset_index(drop=True)

In [92]:
msires = p_map(partial(ms1_issue,df=df,samples=100,burn=10,save_name='ms1_issue'),range(4))

running chain 0 for model ms1_issue

  0%|          | 0/4 [00:00<?, ?it/s]


Adding these covariates:
['v_Intercept']
Adding these covariates:
['z_Intercept']
Adding these covariates:
['t_Intercept']
Adding these covariates:
['a_Intercept']
running chain 1 for model ms1_issue
Adding these covariates:
['v_Intercept']
Adding these covariates:
['z_Intercept']
Adding these covariates:
['t_Intercept']
Adding these covariates:
['a_Intercept']
running chain 2 for model ms1_issue
Adding these covariates:
['v_Intercept']
Adding these covariates:
['z_Intercept']
Adding these covariates:
['t_Intercept']
Adding these covariates:
['a_Intercept']
running chain 3 for model ms1_issue
Adding these covariates:
['v_Intercept']
Adding these covariates:
['z_Intercept']
Adding these covariates:
['t_Intercept']
Adding these covariates:
['a_Intercept']


/opt/conda/lib/python3.8/site-packages/scipy/optimize/optimize.py:2215: RuntimeWarning: invalid value encountered in double_scalars
  tmp2 = (x - v) * (fx - fw)
/opt/conda/lib/python3.8/site-packages/scipy/optimize/optimize.py:2215: RuntimeWarning: invalid value encountered in double_scalars
  tmp2 = (x - v) * (fx - fw)
/opt/conda/lib/python3.8/site-packages/scipy/optimize/optimize.py:2215: RuntimeWarning: invalid value encountered in double_scalars
  tmp2 = (x - v) * (fx - fw)
/opt/conda/lib/python3.8/site-packages/scipy/optimize/optimize.py:2215: RuntimeWarning: invalid value encountered in double_scalars
  tmp2 = (x - v) * (fx - fw)


 -----------------97%----------------  ] 97 of 100 complete in 28.3 sec-                 3%                  ] 3 of 100 complete in 0.8 sec  [-                 3%                  ] 3 of 100 complete in 0.8 sec[-                 3%                  ] 3 of 100 complete in 0.8 sec[-                 5%                  ] 5 of 100 complete in 1.3 sec[-                 5%                  ] 5 of 100 complete in 1.4 sec[-                 5%                  ] 5 of 100 complete in 1.3 sec[-                 5%                  ] 5 of 100 complete in 1.4 sec[--                7%                  ] 7 of 100 complete in 1.9 sec[--                7%                  ] 7 of 100 complete in 2.0 sec[--                7%                  ] 7 of 100 complete in 2.0 sec[--                7%                  ] 7 of 100 complete in 2.0 sec[---               9%                  ] 9 of 100 complete in 2.5 sec[---               9%                  ] 9 of 100 complete in 2.6 sec[---               9%          

In [93]:
%%time
InfData_msi = InferenceDataFromHDDM(msires, nppc = 10, save_name="msi4")

start converting msi4


  0%|          | 0/4 [00:00<?, ?it/s]

started to generate posterior predicitves
started to generate posterior predicitves
started to generate posterior predicitves
started to generate posterior predicitves
 [----------       28%                  ] 4 of 14 complete in 7.0 sec[----------       28%                  ] 4 of 14 complete in 7.3 sec[----------       28%                  ] 4 of 14 complete in 7.2 sec[----------       28%                  ] 4 of 14 complete in 7.3 sec[-------------    35%                  ] 5 of 14 complete in 13.9 sec[-------------    35%                  ] 5 of 14 complete in 14.1 sec[-------------    35%                  ] 5 of 14 complete in 14.2 sec[-------------    35%                  ] 5 of 14 complete in 14.2 sec[---------------- 42%                  ] 6 of 14 complete in 20.2 sec[---------------- 42%                  ] 6 of 14 complete in 20.6 sec[---------------- 42%                  ] 6 of 14 complete in 20.5 sec[---------------- 42%                  ] 6 of 14 complete in 20.5 sec[------

  0%|          | 0/4 [00:00<?, ?it/s]

started to generate posterior predicitves
started to generate posterior predicitves
started to generate posterior predicitves
started to generate posterior predicitves
 [----------       28%                  ] 4 of 14 complete in 4.5 sec[----------       28%                  ] 4 of 14 complete in 4.6 sec[----------       28%                  ] 4 of 14 complete in 4.6 sec[----------       28%                  ] 4 of 14 complete in 4.6 sec[-------------    35%                  ] 5 of 14 complete in 9.1 sec[-------------    35%                  ] 5 of 14 complete in 9.1 sec[-------------    35%                  ] 5 of 14 complete in 9.1 sec[-------------    35%                  ] 5 of 14 complete in 9.2 sec[---------------- 42%                  ] 6 of 14 complete in 13.7 sec[---------------- 42%                  ] 6 of 14 complete in 13.7 sec[---------------- 42%                  ] 6 of 14 complete in 13.7 sec[---------------- 42%                  ] 6 of 14 complete in 13.9 sec[----------

In [94]:
msires = Parallel(n_jobs = 4)(delayed(ms1_issue)(id = i, df = df, samples=100,burn = 10,thin=2, save_name = 'msi' ) for i in range(4))

In [95]:
%%time
InfData_msi = InferenceDataFromHDDM(msires, nppc = 10, save_name="msi3")

start converting msi3


  0%|          | 0/4 [00:00<?, ?it/s]

started to generate posterior predicitves
started to generate posterior predicitves
started to generate posterior predicitves
started to generate posterior predicitves


Process ForkPoolWorker-219:
Process ForkPoolWorker-220:
Process ForkPoolWorker-223:
Process ForkPoolWorker-212:
Process ForkPoolWorker-215:
Process ForkPoolWorker-213:
Process ForkPoolWorker-210:
Process ForkPoolWorker-224:
  File "/opt/conda/lib/python3.8/site-packages/multiprocess/synchronize.py", line 101, in __enter__
    return self._semlock.__enter__()
Process ForkPoolWorker-216:
Process ForkPoolWorker-211:
Process ForkPoolWorker-218:
Process ForkPoolWorker-217:
Process ForkPoolWorker-209:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-214:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/multiprocess/process.py", line 315, in _bootstrap
    self.run()
Traceback

KeyboardInterrupt: 

  File "/opt/conda/lib/python3.8/site-packages/multiprocess/connection.py", line 219, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/opt/conda/lib/python3.8/site-packages/multiprocess/pool.py", line 114, in worker
    task = get()
  File "/opt/conda/lib/python3.8/site-packages/multiprocess/queues.py", line 358, in get
    with self._rlock:
  File "/opt/conda/lib/python3.8/site-packages/hddm/models/hddm_regression.py", line 52, in random
    sampled_rts.loc[i, 'rt'] = hddm.utils.flip_errors(samples).rt.iloc[0]
KeyboardInterrupt
  File "/opt/conda/lib/python3.8/site-packages/multiprocess/connection.py", line 417, in _recv_bytes
    buf = self._recv(4)
  File "/opt/conda/lib/python3.8/site-packages/multiprocess/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.8/site-packages/multiprocess/connection.py", line 382, in _recv
    chunk = read(handle, remaining)
  File "/opt/conda/lib/python3.8/site-packages/multiprocess/s